In [204]:
import time
import math
import re
import random
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
import undetected_chromedriver as uc

In [200]:
# Fonction pour extraire les nombres et calculer les pages
def extraire_infos(texte):
    """
    Extrait le nombre de commentaires par page, le nombre total de commentaires,
    et calcule le nombre de pages à partir d'un texte donné.
    """
    chiffres = [int(s) for s in re.findall(r'\d+', texte)]
    if len(chiffres) >= 2:
        nb_commentaires_par_page = chiffres[1]  # Exemple : "15" (2e chiffre)
        nb_total_commentaires = chiffres[-1]   # Exemple : "747" (dernier chiffre)
        nb_pages = math.ceil(nb_total_commentaires / nb_commentaires_par_page)
        return nb_commentaires_par_page, nb_total_commentaires, nb_pages
    else:
        return None, None, None

# Fonction pour scraper les avis d'une page
def scraper_page(driver):
    """
    Récupère les avis d'une seule page.
    """
    data = []
    # Récupération des éléments sur la page
    pseudos = driver.find_elements(By.XPATH, "//span[@class='biGQs _P fiohW fOtGX']")
    titres = driver.find_elements(By.XPATH, "//div[@class='biGQs _P fiohW qWPrE ncFvv fOtGX']")
    etoiles = driver.find_elements(By.XPATH, "//div[@class='OSBmi J k']")
    nb_etoiles = [re.search(r'(\d+),', etoile.get_attribute("textContent")).group(1) for etoile in etoiles]
    dates = [driver.execute_script("return arguments[0].childNodes[0].textContent;", elem).strip() for elem in driver.find_elements(By.XPATH, "//div[@class='aVuQn']")]
    experiences = driver.find_elements(By.XPATH, "//span[@class='DlAxN']")
    reviews = driver.find_elements(By.XPATH, "//div[@data-test-target='review-body']//span[@class='JguWG' and not(ancestor::div[contains(@class, 'csNQI')])]")

    for i in range(len(titres)):
        avis = {
            "pseudo": pseudos[i].text if i < len(pseudos) else "",
            "titre_review": titres[i].text if i < len(titres) else "",
            "nb_etoiles": nb_etoiles[i] if i < len(nb_etoiles) else "",
            "date": dates[i] if i < len(dates) else "",
            "experience": experiences[i].text if i < len(experiences) else "",
            "review": reviews[i].text if i < len(reviews) else ""
        }
        data.append(avis)
    return data

# Fonction pour scraper les avis de toutes les pages
def scraper_toutes_pages(driver, nb_pages):
    """
    Scrape les avis de toutes les pages en utilisant la fonction `scraper_page`.
    """
    all_data = []
    actions = ActionChains(driver)
    for page in range(1, nb_pages + 1):
        print(f"Scraping de la page {page}...")
        # Récupérer les données de la page actuelle
        data = scraper_page(driver)
        print(f"Données collectées pour la page {page} : {len(data)} avis")
        all_data.extend(data)
        # Navigation vers la page suivante
        try:
            time.sleep(5)
            next_button = WebDriverWait(driver, 20).until(
                EC.element_to_be_clickable((By.XPATH, "//a[@aria-label='Page suivante']"))
            )
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", next_button)
            
            actions.move_to_element(next_button).click().perform()
            
            # Attendre qu'un élément clé de la page suivante apparaisse
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "//div[@class='biGQs _P fiohW qWPrE ncFvv fOtGX']"))
            )
            print("Page suivante chargée.")
        except Exception as e:
            print("Erreur ou plus de pages :", e)
            break
    return all_data

# Tester les fonctions
def test_scraping(driver, nbPages_texte):
    """
    Teste l'ensemble du processus de scraping :
    - Extraction d'informations sur les pages
    - Scraping des avis sur toutes les pages
    """
    # Étape 1 : Extraire les infos
    nb_commentaires_par_page, nb_total_commentaires, nb_pages = extraire_infos(nbPages_texte)
    print(f"Commentaires par page : {nb_commentaires_par_page}")
    print(f"Total commentaires : {nb_total_commentaires}")
    print(f"Nombre de pages : {nb_pages}")

    # Étape 2 : Scraper les avis sur toutes les pages
    if nb_pages:
        all_data = scraper_toutes_pages(driver, nb_pages)
        print(f"Scraping terminé. Total d'avis collectés : {len(all_data)}")
        return all_data
    else:
        print("Erreur dans le calcul des pages.")
        return []


In [36]:
# Service pour ChromeDriver
# Modifier avec le bon chemin
service = Service('C:/Users/Ihnhn/Desktop/M2 SISE/NLP/Projet/chromedriver.exe')
# Step 3: Rotate user agents 
user_agents = [
    # Add your list of user agents here
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.1 Safari/605.1.15',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 13_1) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.1 Safari/605.1.15',
]
# Configuration du navigateur
options = uc.ChromeOptions()
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("start-maximized")
#options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")

# select random user agent
user_agent = random.choice(user_agents)
# pass in selected user agent as an argument
options.add_argument(f'--user-agent={user_agent}')
# Lancement du navigateur
driver = uc.Chrome(options=options, service=service)

# Ouvrez TripAdvisor
#driver.get("https://www.tripadvisor.fr/Restaurant_Review-g187265-d3727154-Reviews-Les_Terrasses_de_Lyon-Lyon_Rhone_Auvergne_Rhone_Alpes.html")
driver.get("https://www.tripadvisor.fr/Restaurant_Review-g187265-d23110895-Reviews-Frazarin-Lyon_Rhone_Auvergne_Rhone_Alpes.html")
# Exécution de JavaScript pour rendre Selenium indétectable
driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
time.sleep(3)
click_cookies = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR,"button[id='onetrust-reject-all-handler']"))).click()

print("Page ouverte avec un User-Agent réaliste")

Page ouverte avec un User-Agent réaliste


In [ ]:
nbPages_texte = driver.find_element("xpath", "//div[@class='Ci']").text

In [ ]:
data = test_scraping(driver, nbPages_texte)